IPL_Ball_By_Ball as a createOrReplaceTempView data_set1

In [0]:
ipl_ball_by_ball = sqlContext.read.format('csv').option('header', 'true').option('inferSchema',
'true').load('/FileStore/tables/ipl_ball_by_ball.csv')
#ipl_ball_by_ball.printSchema()
#ipl_ball_by_ball.show()
ipl_ball_by_ball.createOrReplaceTempView("data_set1")

IPL_Match as a createOrReplaceTempView data_set2

In [0]:
ipl_matches = sqlContext.read.format('csv').option('header', 'true').option('inferSchema',
'true').load('/FileStore/tables/ipl_matches.csv')
ipl_matches.createOrReplaceTempView("data_set2")

IPL_Venue as a createOrReplaceTempView data_set3

In [0]:
ipl_venue = sqlContext.read.format('csv').option('header', 'true').option('inferSchema',
'true').load('/FileStore/tables/ipl_venue.csv')
ipl_venue.createOrReplaceTempView("data_set3")

A.Write a query to find the highest extra runs given by a team in a match.

In [0]:

%sql
SELECT a.match_id ,b.TotalExtraRuns
FROM data_set2 a 
JOIN 
  ( 
    SELECT SUM(extra_runs) TotalExtraRuns, match_id 
    FROM data_set1
    GROUP BY match_id 
  ) b ON a.match_id = b.match_id
ORDER BY b.TotalExtraRuns 
DESC LIMIT 1; 


match_id,TotalExtraRuns
419132,38


B.Write a query to find the Leading wicket taker in the IPL?

In [0]:
%sql
select distinct bowler,sum(is_wicket) as totalwicket from data_set1 where dismissal_kind <>'runout'
group by bowler order by totalwicket  DESC LIMIT 1;

bowler,totalwicket
SL Malinga,188


C.Write a query to return a report for highest run scorer in matches which were affected by Duckworth-Lewis’s method (D/L method).

In [0]:
%sql
select a.match_id,b.batsman,sum(batsman_runs) dl from data_set2 a inner join data_set1 b on a.match_id=b.match_id where method='D/L' group by a.match_id,b.batsman,a.match_id order by dl desc limit 1;

match_id,batsman,dl
980999,V Kohli,113


D.
Write a query to return a report for highest strike rate by a batsman in powerplay (1-6 overs)

In [0]:
%sql
select batsman,(sum(batsman_runs)/count(batsman_runs))*100 as Strike_rate
from data_set1
where overs between 1 and 6 
group by batsman order by Strike_rate  DESC LIMIT 1;


batsman,Strike_rate
Shahid Afridi,243.75


E. Write a query to return a report for highest extra runs in a venue (stadium, city).

In [0]:
%sql
SELECT a.match_id ,b.TotalExtraRuns,c.venue,c.city
FROM data_set2 a join data_set3 c on a.venue_id=c.venue_id 
JOIN 
  ( 
    SELECT SUM(extra_runs) TotalExtraRuns, match_id 
    FROM data_set1
    GROUP BY match_id 
  ) b ON a.match_id = b.match_id
ORDER BY b.TotalExtraRuns 
DESC LIMIT 1; 

match_id,TotalExtraRuns,venue,city
419132,38,Brabourne Stadium,Mumbai


F. Write a query to return a report for the cricketers with the most number of players of the match award in neutral venues.

In [0]:
%sql
Select player_of_match, count(player_of_match) AS players_of_the_match_award from data_set2 where neutral_venue=0  group by player_of_match  ORDER BY players_of_the_match_award
DESC LIMIT 1;  

--------here is confusion neutral_venue=0 or neutral_venue=1 for zero excute above code for 1 excute below code-----------------------------

Select player_of_match, count(player_of_match) AS players_of_the_match_award from data_set2 where neutral_venue=1  group by player_of_match  ORDER BY players_of_the_match_award
DESC LIMIT 1;  



player_of_match,players_of_the_match_award
YK Pathan,3


G. Write a query to get a list of top 10 players with the highest batting average

In [0]:
%sql
SELECT batsman,
       SUM(batsman_runs) as Total_runs, 
       SUM(batsman_runs)/COUNT(CASE WHEN dismissal_kind <> 'NA' THEN 1 END) as Average
FROM data_set1 
GROUP BY batsman
ORDER BY Average DESC LIMIT 10; 

batsman,Total_runs,Average
Iqbal Abdulla,88,88.0
RD Gaikwad,204,68.0
MN van Wyk,167,55.666666666666664
PD Collingwood,203,50.75
HM Amla,577,44.38461538461539
LH Ferguson,44,44.0
KL Rahul,2647,42.693548387096776
AB de Villiers,4849,42.53508771929825
DA Warner,5254,41.698412698412696
JP Duminy,2029,41.40816326530612


H. Write a query to find out who has officiated (as an umpire) the most number of matches in IPL.

In [0]:
%sql 
with t as(
select count(match_id) as top,umpire1 from data_set2 group by umpire1 union select count(match_id) as top1,umpire2 from data_set2 group by umpire2 ) select sum(top),umpire1 from t group by umpire1 order by 1 desc limit 1;



sum(top),umpire1
121,S Ravi


I. Find venue details of the match where V Kohli scored his highest individual runs in IPL.

In [0]:
%sql

    SELECT c.venue,sum(batsman_runs) v_kolhi FROM data_set1 a inner join data_set2 b on a.match_id=b.match_id inner join data_set3 c on b.venue_id=c.venue_id where batsman='V Kohli' group by c.venue ORDER BY v_kolhi
DESC LIMIT 1;  

  
   

venue,v_kolhi
M.Chinnaswamy Stadium,2346
